# Modeling Málaga Temperature with Bayesian Inference
This notebook demonstrates how to model average temperature data using a sinusoidal model in a Bayesian framework, implemented with PyMC.

## ⚠️ Data Notice
*The original dataset `temperatura.txt` used in this project is not available. However, the code below remains fully functional with any compatible time series dataset.*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pymc as pm
import os

## Load Temperature Data
Assumes a file `temperatura.txt` with two columns: time and temperature.

In [ ]:
# ⚠️ Commented out due to missing dataset
# current_dir = os.path.dirname(os.path.abspath("temperatura.txt"))
# file_path = os.path.join(current_dir, 'temperatura.txt')
# data = np.loadtxt(file_path)
# x = data[:, 0]
# y = data[:, 1]

## Bayesian Model Definition
We define a sinusoidal model with a constant offset and Gaussian noise.

In [ ]:
with pm.Model() as model:
    amplitude = pm.HalfNormal('amplitude', sigma=1)
    frequency = pm.Normal('frequency', mu=0, sigma=1)
    b = pm.Normal('b', mu=200000, sigma=1)
    noise_sd = pm.HalfNormal('noise_sd', sigma=1)

    y_hat = amplitude * pm.math.sin(2 * np.pi * frequency * x) + b

    y_obs = pm.Normal('y_obs', mu=y_hat, sigma=noise_sd, observed=y)

    trace = pm.sample(2000, tune=1000)

## Posterior Analysis

In [ ]:
pm.plot_trace(trace)
plt.show()

pm.plot_posterior(trace)
plt.show()

## Predictive Mean and Confidence Interval

In [ ]:
amplitude_samples = trace.posterior['amplitude'].values
frequency_samples = trace.posterior['frequency'].values
b_samples = trace.posterior['b'].values

x_new = x[None, None, :]
y_hat = (amplitude_samples[..., None] * np.sin(2 * np.pi * frequency_samples[..., None] * x_new)) + b_samples[..., None]

y_hat_mean = y_hat.mean(axis=(0, 1))
y_hat_std = y_hat.std(axis=(0, 1))

plt.figure(figsize=(10, 5))
plt.plot(x, y, 'o', label='Observed data')
plt.plot(x, y_hat_mean, color='C1', label='Predicted value')
plt.fill_between(x, y_hat_mean - 2*y_hat_std, y_hat_mean + 2*y_hat_std, color='C1', alpha=0.3, label='Confidence interval')
plt.title('Time Series Bayesian Analysis with PyMC')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

## Summary
This notebook models temperature as a sinusoidal function using Bayesian inference. Even though the dataset is missing, the model logic and structure are fully preserved and adaptable.